**Cats And Dogs Image Classifier**

This is a Keras Model with CNN (Convolutional neural network).

Dataset used: Kaggle Cats and Dogs.

This model has a accuracy rate of **96%**


In [ ]:
import os
print(os.getcwd())
os.chdir('./Convolutional_Neural_Networks/dataset')
print(os.getcwd())
os.listdir()

In [ ]:
#Declaration of parameters needed for training and validation
train_data_dir =('training_set')
test_data_dir =('test_set')

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Initialising the CNN
model = Sequential()

# 1st convolution and pooling payer
#Convolution Layer
model.add(Conv2D(32, 
                 (3, 3), 
                 activation = 'relu',
                 padding='same',
                 input_shape = (128, 128, 3)))
#Maxpooling Layer
model.add(MaxPooling2D(pool_size=(2, 2)))


# 2nd convolution and pooling payer
#Convolution Layer
model.add(Conv2D(32, 
                 (3, 3), 
                 padding='same',
                 activation = 'relu'))
#Maxpooling Layer
model.add(MaxPooling2D(pool_size=(2, 2)))


# 3rd convolution and pooling payer
#Convolution Layer
model.add(Conv2D(32, 
                 (3, 3), 
                 padding='same',
                 activation = 'relu'))
#Maxpooling Layer
model.add(MaxPooling2D(pool_size=(2, 2)))


#Flattening
model.add(Flatten())


# Fully Dense connection
model.add(Dense(64, 
                activation='relu'))

model.add(Dropout(p=0.6))

model.add(Dense(64, 
                activation='relu'))

model.add(Dense(64, 
                activation='relu'))

model.add(Dropout(p=0.3))

model.add(Dense(1, 
                activation='sigmoid'))


#Compile the model
adam= Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)             

C:\Users\H.Finch\Anaconda3\envs\ocr\lib\site-packages\ipykernel_launcher.py:43: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.6)`
C:\Users\H.Finch\Anaconda3\envs\ocr\lib\site-packages\ipykernel_launcher.py:51: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.3)`


In [ ]:
#Trainning Agementation
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_directory(train_data_dir,
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

#Test Agementation
test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_directory(test_data_dir,
                                            target_size = (128, 128),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

In [ ]:
epochs = 10
model.fit_generator(training_set,
                          steps_per_epoch = 8000,
                          epochs = epochs,
                          validation_data = test_set,
                          validation_steps = 2000)

In [ ]:
from keras.models import load_model
load_model("./ClassifierModel.h5")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
history = model.history
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Part 3 - Making new predictions
import os

import numpy as np
from keras.preprocessing import image
test_image = image.load_img('./predict/cat_or_dog_1.jpg', 
                            target_size = (128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
    print(prediction)
else:
    prediction = 'cat'
    print(prediction)

In [ ]:
print('Train',model.history.history['acc'][9] * 100)
print('Test',model.history.history['val_acc'][9] * 100)